In [48]:
import fastparquet
import pandas as pd
import snscrape.modules.twitter as sntwitter
import datetime as dt
from tqdm.notebook import tqdm

In [39]:
def get_tweets(concepto, start_date, end_date, max_tweets):

	scrap = sntwitter.TwitterSearchScraper(concepto + ' since:' + start_date + ' until:' + end_date)

	tweets_list = {}

	for i, tweet in tqdm(enumerate(scrap.get_items()), total=max_tweets):
		
		tweets_list[i] = {
			'date': tweet.date,
			'id': tweet.id,
			'rawContent': tweet.rawContent,
			'replyCount': tweet.replyCount,
			'retweetCount': tweet.retweetCount,
			'likeCount': tweet.likeCount,
			'quoteCount': tweet.quoteCount,
			'lang': tweet.lang,
			'place': tweet.place,
			'hashtags': tweet.hashtags,
			'mentionedUsers': tweet.mentionedUsers,
			'user_id': tweet.user.id,
			'user_name': tweet.user.username,
			'user_renderedDescription': tweet.user.renderedDescription,
			'user_join_date': tweet.user.created,
			'user_followers': tweet.user.followersCount,
			'user_location': tweet.user.location,
			'user_verified': tweet.user.verified,
			'inReplyToTweetId': str(tweet.inReplyToTweetId)
		}

		if i > max_tweets:
			break

	df = pd.DataFrame.from_dict(tweets_list, orient='index')
	return df

In [ ]:
dataset_mencion1 = get_tweets('convención constitucional', '2019-11-15', '2020-12-31', 1_000)

In [101]:
dataset_mencion1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100002 entries, 0 to 100001
Data columns (total 24 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   date                      100002 non-null  object 
 1   id                        100002 non-null  int64  
 2   rawContent                100002 non-null  object 
 3   replyCount                100002 non-null  int64  
 4   retweetCount              100002 non-null  int64  
 5   likeCount                 100002 non-null  int64  
 6   quoteCount                100002 non-null  int64  
 7   viewCount                 0 non-null       float64
 8   lang                      100002 non-null  object 
 9   coordinates               3324 non-null    object 
 10  place                     3324 non-null    object 
 11  hashtags                  63839 non-null   object 
 12  mentionedUsers            30988 non-null   object 
 13  user_id                   100002 non-null  i

In [89]:
dataset_mencion1.head()

,date,id,rawContent,replyCount,retweetCount,likeCount,quoteCount,viewCount,lang,coordinates,...,user_name,user_renderedDescription,user_join_date,user_followers,user_location,user_verified,inReplyToTweetId,date_day,date_month,sentiment
0,2022-09-03 23:36:16+00:00,1566208484160536576,15.1 millones de electores chilenos están obli...,3,10,37,1,NaN,es,NaN,...,FernanMartinez,Periodista y mánager de talento. Percepcionist...,2009-05-13 20:16:30+00:00,321175,Bogotá/Key Biscayne,True,NaN,2022-09-03,2022-09,0.0
1,2022-09-03 23:32:04+00:00,1566207427837628417,#Chile Acá podrán seguir todos los datos respe...,0,0,0,0,NaN,es,NaN,...,JanusNoticias,Noticias de Uruguay y el mundo. Comunicación r...,2022-05-06 00:08:36+00:00,772,NaN,False,NaN,2022-09-03,2022-09,0.0
2,2022-09-03 23:12:21+00:00,1566202468878503936,@Rodolfo62949483 @Dany41135077 De aprobar o no...,1,1,1,0,NaN,es,NaN,...,joseantoniogor,ANTICOMUNISTA 🇨🇱/Gay pero NO PROGRE ni del Col...,2022-07-05 21:03:56+00:00,13170,NaN,False,1.566200e+18,2022-09-03,2022-09,0.0
3,2022-09-03 23:07:47+00:00,1566201318741385216,@bdelamaza Que más se podía esperar de un FACH...,0,0,0,0,NaN,es,NaN,...,David13364452,soltero hasta decir basta.. 😏,2021-11-21 17:17:12+00:00,123,NaN,False,1.565893e+18,2022-09-03,2022-09,0.0
4,2022-09-03 23:05:51+00:00,1566200833225678850,Este domingo 4 de septiembre se llevará a cabo...,0,0,0,0,NaN,es,NaN,...,RadiochevereC,Somos La Primera Radio de Venezolanos en Chile...,2019-10-21 21:08:32+00:00,60,Santiago de Chile,False,NaN,2022-09-03,2022-09,0.0


In [ ]:
# dataset_mencion1.to_parquet('datasets/dataset_mencion1.parquet', index=False, engine='fastparquet', compression='gzip')

In [70]:
dataset_mencion1.to_csv('datasets/dataset_mencion1.csv', index=False)